# Visualisations of attention (CNN-LSTM) Model results

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import json
from pathlib import Path
import os
import numpy as np
sns.set_style("darkgrid")
sns.set_theme("paper")

## Load in Experiment Data from json

In [ ]:
POLYN_DECAY_RES_PATH = os.path.join(Path(os.path.abspath("")), "polynomial_decay_results.json")

with open(POLYN_DECAY_RES_PATH) as f:
    poly_results = json.load(f)

In [ ]:
CONST_RES_PATH = os.path.join(Path(os.path.abspath("")), "attention_const_sparsity_results.json")

with open(CONST_RES_PATH) as f:
    const_res_temp = json.load(f)

## Reformat Results Dict

In [ ]:
pruning_percentages =[]
const_results = {"baseline": {"aucs":[], "inf_time": [], "size": []},
           "int8": {"aucs":[], "inf_time": [], "size": []},
           "float16": {"aucs":[], "inf_time": [], "size": []}
}

for percentage, result in const_res_temp.items():
    pruning_percentages.append(float(percentage))
    
    const_results["baseline"]["aucs"].append(float(result["default"]["auc"]))
    const_results["baseline"]["inf_time"].append(float(result["default"]["inf_time"]))
    const_results["baseline"]["size"].append(float(result["default"]["size"]))
    
    const_results["int8"]["aucs"].append(float(result["int8"]["auc"]))
    const_results["int8"]["inf_time"].append(float(result["int8"]["inf_time"]))
    const_results["int8"]["size"].append(float(result["int8"]["size"]))
    
    const_results["float16"]["aucs"].append(float(result["float16"]["auc"]))
    const_results["float16"]["inf_time"].append(float(result["float16"]["inf_time"]))
    const_results["float16"]["size"].append(float(result["float16"]["size"]))
pruning_percentages = np.array(pruning_percentages) * 100

In [ ]:
results = {"baseline": {"aucs":[], "inf_time": [], "size": []},
           "int8": {"aucs":[], "inf_time": [], "size": []},
           "float16": {"aucs":[], "inf_time": [], "size": []}
}

for percentage, result in poly_results.items():
    
    results["baseline"]["aucs"].append(float(result["default"]["auc"]))
    results["baseline"]["inf_time"].append(float(result["default"]["inf_time"]))
    results["baseline"]["size"].append(float(result["default"]["size"]))
    
    results["int8"]["aucs"].append(float(result["int8"]["auc"]))
    results["int8"]["inf_time"].append(float(result["int8"]["inf_time"]))
    results["int8"]["size"].append(float(result["int8"]["size"]))
    
    results["float16"]["aucs"].append(float(result["float16"]["auc"]))
    results["float16"]["inf_time"].append(float(result["float16"]["inf_time"]))
    results["float16"]["size"].append(float(result["float16"]["size"]))

## Visualisations

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1,3, sharex= True, figsize=(12,3))

print(pruning_percentages)
const_aucs = ax1.plot(pruning_percentages, const_results["baseline"]["aucs"])
poly_aucs = ax1.plot(pruning_percentages, results["baseline"]["aucs"])
ax1.set_ylim([0,1])
ax1.set_ylabel("AUC Score")
ax1.legend([const_aucs, poly_aucs], labels = ["Const.", "Poly. Decay"])

const_sizes = ax2.plot(pruning_percentages, const_results["baseline"]["size"])
poly_sizes = ax2.plot(pruning_percentages, results["baseline"]["size"])
ax2.set_ylabel("Gzipped Model Size (MB)")
ax2.legend([const_sizes, poly_sizes], labels = ["Const.", "Poly. Decay"])


const_inf = ax3.plot(pruning_percentages, np.array(const_results["baseline"]["inf_time"])*1000)
poly_inf = ax3.plot(pruning_percentages, np.array(results["baseline"]["inf_time"])*1000)
ax3.legend([const_inf, poly_inf], labels = ["Const.", "Poly. Decay"])
ax3.set_ylabel("Inference Time (ms)")

fig.text(0.5, 0.0001, 'Pruning Percentage (%)', ha='center')
fig.tight_layout()
plt.savefig(os.path.join(Path(os.path.abspath("")), "figures", "cnn_lstm_pruning_exps_all.pdf"))

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1,3, sharex= True, figsize=(12,3))

int8_aucs = ax1.plot(pruning_percentages, results["int8"]["aucs"])
float16_aucs = ax1.plot(pruning_percentages, results["float16"]["aucs"])

ax1.set_ylim([0,1])
ax1.set_ylabel("AUC Score")
ax1.legend([int8_aucs, float16_aucs], labels = ["Int8", "Float16"])

int8_size = ax2.plot(pruning_percentages, results["int8"]["size"])
float16_size = ax2.plot(pruning_percentages, results["float16"]["size"])
ax2.set_ylabel("Gzipped Model Size (MB)")
ax2.legend([int8_size, float16_size], labels = ["Int8", "Float16"])


int8_inf = ax3.plot(pruning_percentages, np.array(results["int8"]["inf_time"])*1000)
float16_inf = ax3.plot(pruning_percentages, np.array(results["float16"]["inf_time"])*1000)
ax3.set_ylabel("Inference Time (ms)")
ax3.legend([int8_inf, float16_inf], labels = ["Int8", "Float16"])

fig.text(0.5, 0.0001, 'Pruning Percentage (%)', ha='center')
fig.tight_layout()
fig.show()
plt.savefig(os.path.join(Path(os.path.abspath("")), "figures", "cnn_lstm_quant_exps_all.pdf"))

## Print strings for LateX table

In [ ]:
# Sparsity & AUC & Compressed Model Size (MB) & Inference Time (s) & 8-Bit  Model Size & 8-Bit Inference Time & 16-Bit Model Size & 16-Bit Inference Time \\
for percent, results in const_res_temp.items():
    auc = results["default"]["auc"]
    cms = results["default"]["size"]
    it = results["default"]["inf_time"] * 1000
    auc8 = results["int8"]["auc"] 
    cms8 = results["int8"]["size"] 
    it8 = results["int8"]["inf_time"] * 1000
    auc16 = results["float16"]["auc"]
    cms16 = results["float16"]["size"]
    it16 = results["float16"]["inf_time"] * 1000
    string_to_output = f"{float(percent)*100:.1f} & {auc:.3f} & {auc8:.3f} & {auc16:.3f} & {cms:.3f} & {cms8:.3f}  & {cms16:.3f}  & {it:.3f} & {it8:.3f}   & {it16:.3f} \\\\"
    print(string_to_output)

In [ ]:
# Sparsity & AUC & Compressed Model Size (MB) & Inference Time (s) & 8-Bit  Model Size & 8-Bit Inference Time & 16-Bit Model Size & 16-Bit Inference Time \\
for percent, results in poly_results.items():
    auc = results["default"]["auc"]
    cms = results["default"]["size"]
    it = results["default"]["inf_time"] * 1000
    auc8 = results["int8"]["auc"] 
    cms8 = results["int8"]["size"] 
    it8 = results["int8"]["inf_time"] * 1000
    auc16 = results["float16"]["auc"]
    cms16 = results["float16"]["size"]
    it16 = results["float16"]["inf_time"] * 1000
    string_to_output = f"{float(percent)*100:.1f} & {auc:.3f} & {auc8:.3f} & {auc16:.3f} & {cms:.3f} & {cms8:.3f}  & {cms16:.3f}  & {it:.3f} & {it8:.3f}   & {it16:.3f} \\\\"
    print(string_to_output)